In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf
import tensorflow_hub as hub

img_size = 224
num_classes = 7
model_path = 'https://tfhub.dev/sayakpaul/swin_tiny_patch4_window7_224_fe'

inputs = tf.keras.Input((img_size, img_size, 3))
hub_module = hub.KerasLayer(model_path, trainable=False)


x = hub_module(inputs, training=True)
x = tf.keras.layers.Dense(1024, activation="gelu")(x)
x = tf.keras.layers.Dropout(0.25)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath='Swin_pretrained.keras', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=callbacks
)

